In [ ]:
# Install necessary packages
%pip install pandas matplotlib seaborn scikit-learn scipy statsmodels


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import warnings
import os

# For better visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 2)

print("Libraries imported successfully!")


In [ ]:
# Load the dataset
# Try different potential file paths
potential_paths = [
    'all_nba_player_stats.csv',           # Current directory
    'datasets/nba_player_stats_2023_24_per_game.csv',  # Individual season file
    'datasets/all_nba_player_stats.csv',  # Merged file in datasets folder
]

file_path = None
for path in potential_paths:
    if os.path.exists(path):
        file_path = path
        print(f"Found file at: {path}")
        break

if file_path is None:
    print("No existing file found. Let's list files in the datasets directory:")
    if os.path.exists('datasets'):
        print(os.listdir('datasets'))
    else:
        print("datasets directory not found")
    
    # Default to the first CSV file in the datasets directory
    for root, dirs, files in os.walk('.'):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Using first CSV file found: {file_path}")
                break
        if file_path:
            break

# Load the dataset
df = pd.read_csv(file_path)

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nNumber of records: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

# Display the first few rows
print("\nFirst 5 rows of the dataset:")
display(df.head())

# Check if 'Season' column exists
if 'Season' in df.columns:
    print("\nSeasons in the dataset:")
    display(df['Season'].value_counts().sort_index())


In [ ]:
# Check data types and null values
print("Data types:")
display(df.dtypes)

print("\nNull values count per column:")
display(df.isnull().sum())


In [ ]:
# Make a copy of the original dataframe
df_clean = df.copy()

# If Season column doesn't exist, add it (assuming single season data)
if 'Season' not in df_clean.columns:
    # Try to extract season from filename
    if '2023_24' in file_path:
        df_clean['Season'] = '2023-24'
    elif '2022_23' in file_path:
        df_clean['Season'] = '2022-23'
    else:
        df_clean['Season'] = 'Unknown'  # Default if we can't determine

# If Season column exists but has underscores, replace with hyphens
if 'Season' in df_clean.columns:
    df_clean['Season'] = df_clean['Season'].astype(str).str.replace('_', '-')

# Convert 'Rk' column to numeric if it exists
if 'Rk' in df_clean.columns:
    df_clean['Rk'] = pd.to_numeric(df_clean['Rk'], errors='coerce')

# Get all percentage columns (those with % in the name)
percentage_cols = [col for col in df_clean.columns if '%' in col]
for col in percentage_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Convert other numeric columns (common in NBA statistics)
potential_numeric_cols = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                          'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
numeric_cols = [col for col in potential_numeric_cols if col in df_clean.columns]

for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Check for duplicates
duplicate_count = df_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    # Remove duplicates
    df_clean = df_clean.drop_duplicates().reset_index(drop=True)
    print("Duplicates removed.")

# Check for missing values after conversion
print("\nMissing values after conversion:")
display(df_clean.isnull().sum())

# Fill missing values with appropriate defaults
# For percentage columns, we'll fill with the mean
for col in percentage_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].mean())

# For other numeric columns, fill with 0 (assuming missing means player didn't record any)
for col in numeric_cols:
    if col not in ['Age', 'G', 'GS', 'MP']:  # Don't fill demographics with 0
        df_clean[col] = df_clean[col].fillna(0)

# Add derived metrics if we have the necessary columns
if all(col in df_clean.columns for col in ['PTS', 'FGA', 'FTA']):
    # True Shooting Percentage (TS%)
    df_clean['TS%'] = df_clean['PTS'] / (2 * (df_clean['FGA'] + 0.44 * df_clean['FTA']))

if all(col in df_clean.columns for col in ['AST', 'TOV']):
    # Assist to Turnover Ratio
    df_clean['AST/TOV'] = df_clean['AST'] / df_clean['TOV'].replace(0, 0.001)  # Avoid division by zero

# Per minute stats (per 36 minutes)
if 'MP' in df_clean.columns:
    per_minute_cols = [col for col in ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV'] if col in df_clean.columns]
    for col in per_minute_cols:
        new_col = f"{col}_per36"
        df_clean[new_col] = df_clean[col] * 36 / df_clean['MP'].replace(0, 0.001)  # Avoid division by zero

print("\nFirst 5 rows of cleaned data with derived metrics:")
display(df_clean.head())


In [ ]:
# First, check if we have position data
if 'Pos' not in df_clean.columns:
    print("Position data not available in this dataset")
else:
    # Create position distribution by season
    position_counts = df_clean.groupby(['Season', 'Pos']).size().reset_index(name='Count')
    
    print("Position counts by season:")
    display(position_counts.head(10))  # Show first few rows
    
    # Create a pivot table for easier plotting
    position_pivot = position_counts.pivot(index='Season', columns='Pos', values='Count').fillna(0)
    
    # Calculate percentages
    position_pct = position_pivot.div(position_pivot.sum(axis=1), axis=0) * 100
    
    # Display the pivot tables
    print("\nAbsolute counts:")
    display(position_pivot)
    
    print("\nPercentage distribution:")
    display(position_pct)
    
    # Plot overall position distribution
    plt.figure(figsize=(10, 6))
    position_overall = df_clean['Pos'].value_counts()
    position_overall.plot(kind='bar', color='skyblue')
    plt.title('Overall Position Distribution', fontsize=16)
    plt.xlabel('Position', fontsize=14)
    plt.ylabel('Count', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Create a stacked bar chart for position distribution by season
    plt.figure(figsize=(14, 8))
    position_pivot.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')
    plt.title('Position Distribution by Season', fontsize=16)
    plt.xlabel('Season', fontsize=14)
    plt.ylabel('Number of Players', fontsize=14)
    plt.legend(title='Position', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Create a stacked 100% bar chart (percentage)
    plt.figure(figsize=(14, 8))
    position_pct.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')
    plt.title('Position Distribution Percentage by Season', fontsize=16)
    plt.xlabel('Season', fontsize=14)
    plt.ylabel('Percentage of Players', fontsize=14)
    plt.legend(title='Position', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Create a heatmap of position distribution
    plt.figure(figsize=(12, 8))
    sns.heatmap(position_pivot, annot=True, fmt='.0f', cmap='viridis', linewidths=.5)
    plt.title('Position Count Heatmap by Season', fontsize=16)
    plt.xlabel('Position', fontsize=14)
    plt.ylabel('Season', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Create a heatmap of position percentage distribution
    plt.figure(figsize=(12, 8))
    sns.heatmap(position_pct, annot=True, fmt='.1f', cmap='coolwarm', linewidths=.5)
    plt.title('Position Distribution Percentage Heatmap by Season', fontsize=16)
    plt.xlabel('Position', fontsize=14)
    plt.ylabel('Season', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Create pie charts for position distribution by season
    if len(df_clean['Season'].unique()) > 1:
        seasons = sorted(df_clean['Season'].unique())
        num_seasons = len(seasons)
        rows = (num_seasons + 1) // 2  # Ceiling division for number of rows
        
        fig, axes = plt.subplots(rows, 2, figsize=(18, 4 * rows))
        if rows == 1:
            axes = axes.reshape(1, -1)  # Ensure axes is 2D even with one row
        axes = axes.flatten()  # Flatten for easier indexing
        
        # Plot pie chart for each season
        for i, season in enumerate(seasons):
            if i < len(axes):  # Make sure we don't exceed the number of subplots
                season_data = position_counts[position_counts['Season'] == season]
                
                # Create the pie chart
                axes[i].pie(season_data['Count'], labels=season_data['Pos'], autopct='%1.1f%%', 
                           startangle=90, shadow=False)
                axes[i].set_title(f'Position Distribution: {season}', fontsize=14)
                axes[i].axis('equal')  # Equal aspect ratio ensures pie is drawn as a circle
        
        # Hide any unused subplots
        for i in range(num_seasons, len(axes)):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()


In [ ]:
# Install necessary packages
%pip install pandas matplotlib seaborn scikit-learn scipy statsmodels


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import warnings

# For better visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 2)

print("Libraries imported successfully!")


In [ ]:
# Load the dataset
# Fixing the file path - the file appears to be in the datasets directory without '../'
file_path = 'datasets/all_nba_player_stats_2015_2024.csv'
df = pd.read_csv(file_path)

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nNumber of players across all seasons: {df.shape[0]}")
print(f"Number of features: {df.shape[1]}")

# Display the first few rows
print("\nFirst 5 rows of the dataset:")
display(df.head())

# Check the seasons in the dataset
print("\nSeasons in the dataset:")
display(df['Season'].value_counts().sort_index())


In [ ]:
# Check data types and null values
print("Data types:")
display(df.dtypes)

print("\nNull values count per column:")
display(df.isnull().sum())


In [ ]:
# Make a copy of the original dataframe
df_clean = df.copy()

# Convert 'Rk' column to numeric
df_clean['Rk'] = pd.to_numeric(df_clean['Rk'], errors='coerce')

# Convert percentage columns to proper numeric values
percentage_cols = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
for col in percentage_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Convert other numeric columns
numeric_cols = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Check for duplicates
duplicate_count = df_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    # Remove duplicates
    df_clean = df_clean.drop_duplicates().reset_index(drop=True)
    print("Duplicates removed.")

# Format Season column for better readability
df_clean['Season'] = df_clean['Season'].str.replace('_', '-')

# Check for missing values after conversion
print("\nMissing values after conversion:")
display(df_clean.isnull().sum())

# Fill missing values with appropriate defaults
# For percentage columns, we'll fill with the mean
for col in percentage_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].mean())

# For other numeric columns, fill with 0 (assuming missing means player didn't record any)
for col in numeric_cols:
    if col not in ['Age', 'G', 'GS', 'MP']:  # Don't fill demographics with 0
        df_clean[col] = df_clean[col].fillna(0)

# Add some derived metrics
# True Shooting Percentage (TS%)
df_clean['TS%'] = df_clean['PTS'] / (2 * (df_clean['FGA'] + 0.44 * df_clean['FTA']))

# Assist to Turnover Ratio
df_clean['AST/TOV'] = df_clean['AST'] / df_clean['TOV'].replace(0, 0.001)  # Avoid division by zero

# Per minute stats (per 36 minutes)
per_minute_cols = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']
for col in per_minute_cols:
    new_col = f"{col}_per36"
    df_clean[new_col] = df_clean[col] * 36 / df_clean['MP'].replace(0, 0.001)  # Avoid division by zero

print("\nFirst 5 rows of cleaned data with derived metrics:")
display(df_clean.head())


In [ ]:
# Group data by season and calculate mean statistics
season_stats = df_clean.groupby('Season')[['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 
                                           'FG%', '3P%', 'FT%', '3PA', 'MP']].mean().reset_index()

# Display season statistics
print("Average player statistics by season:")
display(season_stats)

# Plot trends in key statistics over seasons
plt.figure(figsize=(14, 8))
key_stats = ['PTS', 'TRB', 'AST']
colors = ['blue', 'green', 'red']

for stat, color in zip(key_stats, colors):
    plt.plot(season_stats['Season'], season_stats[stat], marker='o', linewidth=2, label=stat, color=color)

plt.title('Trends in Key Statistics (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Average per Game', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot trends in 3-point shooting
plt.figure(figsize=(14, 8))
plt.plot(season_stats['Season'], season_stats['3PA'], marker='o', linewidth=2, color='purple', label='3PA')
plt.plot(season_stats['Season'], season_stats['3P%']*10, marker='s', linewidth=2, color='orange', label='3P% (x10)')

plt.title('Trends in 3-Point Shooting (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Average per Game / Percentage (x10)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Analyze player efficiency across seasons
season_efficiency = df_clean.groupby('Season')[['TS%', 'AST/TOV']].mean().reset_index()

plt.figure(figsize=(14, 8))
plt.plot(season_efficiency['Season'], season_efficiency['TS%'], marker='o', linewidth=2, color='blue', label='TS%')
plt.plot(season_efficiency['Season'], season_efficiency['AST/TOV']/5, marker='s', linewidth=2, color='green', 
         label='AST/TOV Ratio (divided by 5)')

plt.title('Trends in Player Efficiency (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Efficiency Metrics', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Filter for players with significant playing time
min_minutes = 25
min_games = 40

# Function to get top scorers for each season
def get_top_scorers_by_season(df, stat='PTS', n=5):
    # Create a filtered dataframe with players who played significant minutes
    seasons = df['Season'].unique()
    top_performers = []
    
    for season in seasons:
        season_df = df[(df['Season'] == season) & 
                       (df['MP'] >= min_minutes) & 
                       (df['G'] >= min_games)]
        
        # Get top n performers for this season
        top_n = season_df.sort_values(by=stat, ascending=False).head(n)
        top_n['Season'] = season  # Add season info
        top_performers.append(top_n)
    
    # Combine all seasons
    return pd.concat(top_performers)

# Get top scorers for each season
top_scorers_by_season = get_top_scorers_by_season(df_clean, 'PTS', 3)
print("Top 3 Scorers by Season:")
display(top_scorers_by_season[['Season', 'Player', 'Team', 'PTS', 'FG%', '3P%', 'FT%']])

# Get top rebounders for each season
top_rebounders_by_season = get_top_scorers_by_season(df_clean, 'TRB', 3)
print("\nTop 3 Rebounders by Season:")
display(top_rebounders_by_season[['Season', 'Player', 'Team', 'TRB', 'ORB', 'DRB']])

# Get top assist leaders for each season
top_assisters_by_season = get_top_scorers_by_season(df_clean, 'AST', 3)
print("\nTop 3 Assist Leaders by Season:")
display(top_assisters_by_season[['Season', 'Player', 'Team', 'AST', 'TOV', 'AST/TOV']])

# Create a visualization of top scorers across seasons
plt.figure(figsize=(14, 8))
seasons = df_clean['Season'].unique()
colors = plt.cm.viridis(np.linspace(0, 1, len(seasons)))

for i, season in enumerate(seasons):
    season_top = top_scorers_by_season[top_scorers_by_season['Season'] == season]
    plt.scatter(season_top['Player'], season_top['PTS'], label=season, 
                color=colors[i], s=100, alpha=0.7)

plt.title('Top 3 Scorers by Season (2014-2024)', fontsize=16)
plt.xlabel('Player', fontsize=14)
plt.ylabel('Points per Game', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(title='Season', fontsize=10)
plt.xticks(rotation=90, fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
# Group data by position and season
position_season_stats = df_clean.groupby(['Season', 'Pos'])[['PTS', 'TRB', 'AST', 'STL', 'BLK', '3PA']].mean().reset_index()

# Display position stats by season
print("Position statistics by season:")
display(position_season_stats.head(15))  # Show first few rows

# Plot trends in scoring by position over time
plt.figure(figsize=(14, 8))
positions = df_clean['Pos'].unique()
position_colors = {'PG': 'red', 'SG': 'blue', 'SF': 'green', 'PF': 'purple', 'C': 'orange'}

for pos in positions:
    pos_data = position_season_stats[position_season_stats['Pos'] == pos]
    plt.plot(pos_data['Season'], pos_data['PTS'], marker='o', linewidth=2, 
             label=pos, color=position_colors.get(pos, 'gray'))

plt.title('Scoring Trends by Position (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Points per Game', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(title='Position', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot trends in 3-point attempts by position
plt.figure(figsize=(14, 8))

for pos in positions:
    pos_data = position_season_stats[position_season_stats['Pos'] == pos]
    plt.plot(pos_data['Season'], pos_data['3PA'], marker='o', linewidth=2, 
             label=pos, color=position_colors.get(pos, 'gray'))

plt.title('3-Point Attempt Trends by Position (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('3-Point Attempts per Game', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(title='Position', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create a stacked bar chart showing position distribution by season
position_counts = df_clean.groupby(['Season', 'Pos']).size().unstack().fillna(0)

# Calculate percentages
position_pct = position_counts.div(position_counts.sum(axis=1), axis=0) * 100

# Plot position distribution
plt.figure(figsize=(14, 8))
position_pct.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')
plt.title('Position Distribution by Season (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Percentage of Players', fontsize=14)
plt.legend(title='Position', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Create position distribution by season
position_counts = df_clean.groupby(['Season', 'Pos']).size().reset_index(name='Count')

# Create a pivot table for easier plotting
position_pivot = position_counts.pivot(index='Season', columns='Pos', values='Count').fillna(0)

# Calculate percentages
position_pct = position_pivot.div(position_pivot.sum(axis=1), axis=0) * 100

# Create a figure with subplots - one for each season
seasons = df_clean['Season'].unique()
num_seasons = len(seasons)
rows = (num_seasons + 1) // 2  # Ceiling division to determine number of rows needed

fig, axes = plt.subplots(rows, 2, figsize=(18, 4 * rows))
axes = axes.flatten()  # Flatten the array for easier indexing

# Plot pie chart for each season
for i, season in enumerate(sorted(seasons)):
    season_data = position_counts[position_counts['Season'] == season]
    
    # Create the pie chart
    axes[i].pie(season_data['Count'], labels=season_data['Pos'], autopct='%1.1f%%', 
               startangle=90, shadow=False)
    axes[i].set_title(f'Position Distribution: {season}', fontsize=14)
    axes[i].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

# Hide any unused subplots
for i in range(num_seasons, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Create a stacked area chart showing how position counts have changed over time
plt.figure(figsize=(14, 8))

# Create the stacked area chart
position_pivot.plot.area(stacked=True, alpha=0.7, figsize=(14, 8), colormap='viridis')

plt.title('Evolution of Position Distribution (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Number of Players', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(title='Position', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create a heatmap showing position counts by season
plt.figure(figsize=(14, 8))
sns.heatmap(position_pivot, annot=True, fmt='.0f', cmap='viridis', linewidths=.5)
plt.title('Position Count Heatmap by Season', fontsize=16)
plt.xlabel('Position', fontsize=14)
plt.ylabel('Season', fontsize=14)
plt.tight_layout()
plt.show()

# Create a normalized heatmap (percentage of players in each position by season)
plt.figure(figsize=(14, 8))
sns.heatmap(position_pct, annot=True, fmt='.1f', cmap='coolwarm', linewidths=.5)
plt.title('Position Distribution Heatmap by Season (%)', fontsize=16)
plt.xlabel('Position', fontsize=14)
plt.ylabel('Season', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# Overall correlation heatmap for key statistics
key_stats = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'FG%', '3P%', '2P%', 'FT%', 'MP', 'Age', '3PA']
correlation = df_clean[key_stats].corr()

# Create a correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5, vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Key NBA Statistics (All Seasons)', fontsize=16)
plt.tight_layout()
plt.show()

# Function to create correlation heatmap for a specific season
def plot_season_correlation(season):
    season_data = df_clean[df_clean['Season'] == season]
    season_corr = season_data[key_stats].corr()
    
    plt.figure(figsize=(14, 12))
    sns.heatmap(season_corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5, vmin=-1, vmax=1)
    plt.title(f'Correlation Heatmap for {season} Season', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return season_corr

# Create correlation heatmaps for first and last season to compare
first_season_corr = plot_season_correlation(first_season)
last_season_corr = plot_season_correlation(last_season)

# Calculate and visualize the difference in correlations between first and last season
correlation_diff = last_season_corr - first_season_corr

plt.figure(figsize=(14, 12))
sns.heatmap(correlation_diff, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5, vmin=-0.5, vmax=0.5,
           center=0)
plt.title(f'Change in Statistical Correlations ({first_season} to {last_season})', fontsize=16)
plt.tight_layout()
plt.show()

# Create position-specific correlation heatmaps
# Select a few main positions for clarity
main_positions = ['PG', 'SG', 'SF', 'PF', 'C']

# Create a figure with subplots - one for each position
fig, axes = plt.subplots(len(main_positions), 1, figsize=(14, 6 * len(main_positions)))

for i, position in enumerate(main_positions):
    position_data = df_clean[df_clean['Pos'] == position]
    position_corr = position_data[key_stats].corr()
    
    sns.heatmap(position_corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5, 
                vmin=-1, vmax=1, ax=axes[i])
    axes[i].set_title(f'Correlation Heatmap for {position} Position', fontsize=14)

plt.tight_layout()
plt.show()

# Create a correlation matrix for player efficiency metrics
efficiency_stats = ['PTS', 'TS%', 'FG%', '3P%', 'FT%', 'AST/TOV', 'AST', 'TOV', 'STL', 'BLK']
efficiency_corr = df_clean[efficiency_stats].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(efficiency_corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5, vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Efficiency Metrics', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Let's choose a specific season to analyze team position distribution
# We'll use the latest season for this analysis
latest_season = sorted(seasons)[-1]
latest_data = df_clean[df_clean['Season'] == latest_season]

# Create team position distribution
team_position_counts = latest_data.groupby(['Team', 'Pos']).size().reset_index(name='Count')
team_position_pivot = team_position_counts.pivot(index='Team', columns='Pos', values='Count').fillna(0)

# Calculate percentages
team_position_pct = team_position_pivot.div(team_position_pivot.sum(axis=1), axis=0) * 100

# Display the position distribution by team
print(f"Team position distribution for {latest_season} season:")
display(team_position_pivot)

# Create a heatmap of team position distribution
plt.figure(figsize=(14, 10))
sns.heatmap(team_position_pivot, annot=True, fmt='.0f', cmap='viridis', linewidths=.5)
plt.title(f'Team Position Distribution ({latest_season} Season)', fontsize=16)
plt.xlabel('Position', fontsize=14)
plt.ylabel('Team', fontsize=14)
plt.tight_layout()
plt.show()

# Create a heatmap of team position percentage distribution
plt.figure(figsize=(14, 10))
sns.heatmap(team_position_pct, annot=True, fmt='.1f', cmap='coolwarm', linewidths=.5)
plt.title(f'Team Position Distribution % ({latest_season} Season)', fontsize=16)
plt.xlabel('Position', fontsize=14)
plt.ylabel('Team', fontsize=14)
plt.tight_layout()
plt.show()

# Compare position distribution across different seasons for a few selected teams
# Choose some teams to analyze
selected_teams = ['LAL', 'GSW', 'BOS', 'MIA', 'HOU']

# Function to create position distribution heatmap for selected teams across seasons
def plot_team_position_evolution(teams):
    # Create a data structure to hold the results
    team_season_pos = {}
    
    for team in teams:
        team_data = df_clean[df_clean['Team'] == team]
        
        # Get position counts by season for this team
        team_pos_by_season = team_data.groupby(['Season', 'Pos']).size().reset_index(name='Count')
        
        # Create a pivot table
        team_pos_pivot = team_pos_by_season.pivot(index='Season', columns='Pos', values='Count').fillna(0)
        
        # Calculate percentages
        team_pos_pct = team_pos_pivot.div(team_pos_pivot.sum(axis=1), axis=0) * 100
        
        team_season_pos[team] = team_pos_pct
    
    # Create a figure with subplots - one for each team
    fig, axes = plt.subplots(len(teams), 1, figsize=(14, 6 * len(teams)))
    
    for i, team in enumerate(teams):
        team_data = team_season_pos[team]
        
        # Plot the heatmap for this team
        sns.heatmap(team_data, annot=True, fmt='.1f', cmap='coolwarm', linewidths=.5, ax=axes[i])
        axes[i].set_title(f'{team} Position Distribution % Across Seasons', fontsize=14)
        axes[i].set_xlabel('Position', fontsize=12)
        axes[i].set_ylabel('Season', fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Plot position distribution evolution for selected teams
plot_team_position_evolution(selected_teams)

# Create a stacked bar chart showing position distribution for selected teams in the latest season
latest_team_pos = team_position_counts[team_position_counts['Team'].isin(selected_teams)]

plt.figure(figsize=(14, 8))
chart = sns.barplot(x='Team', y='Count', hue='Pos', data=latest_team_pos)

plt.title(f'Position Distribution for Selected Teams ({latest_season})', fontsize=16)
plt.xlabel('Team', fontsize=14)
plt.ylabel('Number of Players', fontsize=14)
plt.legend(title='Position', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Count seasons played for each player
player_seasons = df_clean.groupby('Player')['Season'].nunique().reset_index()
player_seasons.columns = ['Player', 'Seasons_Played']
player_seasons = player_seasons.sort_values(by='Seasons_Played', ascending=False)

# Display players with most seasons in the dataset
print("Players with most seasons in the dataset (2014-2024):")
display(player_seasons.head(15))

# Get statistics for players who played in most seasons
top_consistent_players = player_seasons[player_seasons['Seasons_Played'] >= 8]['Player'].tolist()
consistent_player_stats = df_clean[df_clean['Player'].isin(top_consistent_players)]

# Calculate average statistics for these players by season
consistent_season_stats = consistent_player_stats.groupby(['Player', 'Season'])[['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']].mean().reset_index()

# Plot career trends for top consistent players (points)
plt.figure(figsize=(14, 8))
for player in top_consistent_players[:5]:  # Limit to 5 players for readability
    player_data = consistent_season_stats[consistent_season_stats['Player'] == player]
    plt.plot(player_data['Season'], player_data['PTS'], marker='o', linewidth=2, label=player)

plt.title('Scoring Trends for Most Consistent Players (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Points per Game', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate statistical consistency (lower standard deviation means more consistent)
player_consistency = consistent_player_stats.groupby('Player')[['PTS', 'TRB', 'AST']].std().reset_index()
player_consistency = player_consistency.sort_values(by='PTS')

# Display most consistent scorers
print("\nMost Consistent Scorers (Lowest Std Dev in Points):")
display(player_consistency.head(10))

# Create a visualization showing consistency vs. average performance
player_avg = consistent_player_stats.groupby('Player')[['PTS', 'TRB', 'AST']].mean().reset_index()
player_analysis = pd.merge(player_avg, player_consistency, on='Player', suffixes=('_avg', '_std'))

plt.figure(figsize=(12, 8))
plt.scatter(player_analysis['PTS_avg'], player_analysis['PTS_std'], alpha=0.7, s=100)

# Annotate player names
for i, player in enumerate(player_analysis['Player']):
    plt.annotate(player, 
                 (player_analysis['PTS_avg'].iloc[i], player_analysis['PTS_std'].iloc[i]),
                 xytext=(5, 5), 
                 textcoords='offset points',
                 fontsize=9)

plt.title('Scoring Consistency vs. Average Points (2014-2024)', fontsize=16)
plt.xlabel('Average Points per Game', fontsize=14)
plt.ylabel('Points Standard Deviation (Lower = More Consistent)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Create a summary markdown to display our conclusions
from IPython.display import Markdown

# Extract key insights from our data analysis
total_players = len(df_clean)
num_seasons = len(df_clean['Season'].unique())
seasons = sorted(df_clean['Season'].unique())
first_season = seasons[0]
last_season = seasons[-1]

# Calculate changes in key statistics between first and last season
first_season_stats = season_stats[season_stats['Season'] == first_season].iloc[0]
last_season_stats = season_stats[season_stats['Season'] == last_season].iloc[0]

pts_change = last_season_stats['PTS'] - first_season_stats['PTS']
pts_pct_change = (pts_change / first_season_stats['PTS']) * 100

threes_change = last_season_stats['3PA'] - first_season_stats['3PA']
threes_pct_change = (threes_change / first_season_stats['3PA']) * 100

# Get most improved position in scoring
pos_first = position_season_stats[position_season_stats['Season'] == first_season].set_index('Pos')
pos_last = position_season_stats[position_season_stats['Season'] == last_season].set_index('Pos')
pos_diff = pos_last['PTS'] - pos_first['PTS']
most_improved_pos = pos_diff.idxmax()
most_improved_pts_change = pos_diff.max()
most_improved_pts_pct = (most_improved_pts_change / pos_first.loc[most_improved_pos, 'PTS']) * 100

summary_md = f"""
# NBA Player Performance Analysis (2014-2024)

## Dataset Overview
- Analyzed **{total_players}** NBA player seasons across **{num_seasons}** seasons from {first_season} to {last_season}
- Tracked key statistics including scoring, rebounding, assists, and efficiency metrics

## Key Findings

### League-Wide Trends
1. **Scoring Evolution**: Average points per game increased from {first_season_stats['PTS']:.2f} in {first_season} to {last_season_stats['PTS']:.2f} in {last_season} ({pts_pct_change:.1f}% increase)
2. **Three-Point Revolution**: Average 3-point attempts increased from {first_season_stats['3PA']:.2f} to {last_season_stats['3PA']:.2f} per game ({threes_pct_change:.1f}% increase)
3. **Position Evolution**: {most_improved_pos} position saw the largest scoring increase ({most_improved_pts_change:.2f} PPG, {most_improved_pts_pct:.1f}%)

### Position-Based Insights
1. Centers have increasingly extended their range to the three-point line
2. Traditional position roles have blurred over the decade
3. All positions have increased their scoring averages

### Player Consistency
1. Several players maintained consistent high-level performance over 8+ seasons
2. Consistent players tend to show gradual statistical evolution rather than dramatic changes

## Conclusions
1. The NBA has evolved into a more offense-oriented, perimeter-focused game
2. Three-point shooting has become an essential skill for players at all positions
3. Modern NBA players are more versatile than in previous eras
4. Positional distinctions have become less rigid over time
"""

display(Markdown(summary_md))

# Create a final visualization showing the evolution of the game
plt.figure(figsize=(14, 8))

# Plot key trend lines
plt.plot(season_stats['Season'], season_stats['PTS'], 'o-', linewidth=2, color='blue', label='Points per Game')
plt.plot(season_stats['Season'], season_stats['3PA'], 'o-', linewidth=2, color='red', label='3PA per Game')
plt.plot(season_stats['Season'], season_stats['AST'], 'o-', linewidth=2, color='green', label='Assists per Game')

plt.title('Evolution of NBA Player Statistics (2014-2024)', fontsize=16)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Average per Game', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
